In [1]:
from tensorflow import keras

In [2]:
IMG_SIZE = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


Per prima cosa, proviamo ad utilizzare la rete in **transfer learning**. Per farlo:

* selezioniamo una rete (in questo caso, `MobileNetV3Small`) dal package `applications` di Keras;
* specifichiamo il parametro `weights` ad `imagenet`, in modo da caricare i pesi della rete già addestrata su ImageNet;
* rimuoviamo il layer di classificazione di ImageNet impostando `include_top` a `False`;
* impostiamo l'attributo `trainable` a `False` per evitare di modificare i pesi della rete;
* costruiamo un nuovo modello.

In [3]:
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

# sequential
model = keras.Sequential()
model.add(keras.layers.Lambda(keras.applications.mobilenet_v3.preprocess_input, input_shape=(224, 224, 3)))
model.add(base_model)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))

model.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

model.fit(train_ds, epochs=20, validation_data=val_ds)

Epoch 1/20
1094/1094 [==============================] - 115s 104ms/step - loss: 7.5073 - binary_accuracy: 0.4989 - val_loss: 7.5386 - val_binary_accuracy: 0.5037
Epoch 2/20
1094/1094 [==============================] - 125s 115ms/step - loss: 7.6379 - binary_accuracy: 0.4971 - val_loss: 7.5380 - val_binary_accuracy: 0.5037
Epoch 3/20
1094/1094 [==============================] - 129s 118ms/step - loss: 7.6436 - binary_accuracy: 0.4967 - val_loss: 7.5437 - val_binary_accuracy: 0.5034
Epoch 4/20
1094/1094 [==============================] - 127s 116ms/step - loss: 7.6369 - binary_accuracy: 0.4972 - val_loss: 7.5331 - val_binary_accuracy: 0.5043
Epoch 5/20
1094/1094 [==============================] - 126s 115ms/step - loss: 7.6305 - binary_accuracy: 0.4976 - val_loss: 7.5321 - val_binary_accuracy: 0.5043
Epoch 6/20
1094/1094 [==============================] - 125s 115ms/step - loss: 7.6304 - binary_accuracy: 0.4978 - val_loss: 7.5377 - val_binary_accuracy: 0.5043
Epoch 7/20
1094/1094 [======

Proviamo adesso ad effettuare il **fine tuning**. Per farlo, impostiamo `trainable` a `True`, e compiliamo il modello usando un learning rate molto basso (in questo caso, 0.00001).

In [4]:
base_model.trainable = True

model.compile(
    optimizer=keras.optimizers.SGD(1e-5),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
1094/1094 [==============================] - 353s 320ms/step - loss: 7.6108 - binary_accuracy: 0.4982 - val_loss: 7.5203 - val_binary_accuracy: 0.5047
Epoch 2/10
1094/1094 [==============================] - 352s 321ms/step - loss: 7.6114 - binary_accuracy: 0.4981 - val_loss: 7.5141 - val_binary_accuracy: 0.5047
Epoch 3/10
1094/1094 [==============================] - 350s 320ms/step - loss: 7.6158 - binary_accuracy: 0.4981 - val_loss: 7.4919 - val_binary_accuracy: 0.5038
Epoch 4/10
1094/1094 [==============================] - 351s 321ms/step - loss: 7.6150 - binary_accuracy: 0.4981 - val_loss: 7.5126 - val_binary_accuracy: 0.5041
Epoch 5/10
1094/1094 [==============================] - 354s 324ms/step - loss: 7.6078 - binary_accuracy: 0.4983 - val_loss: 7.5461 - val_binary_accuracy: 0.5047
Epoch 6/10
1094/1094 [==============================] - 348s 318ms/step - loss: 7.5951 - binary_accuracy: 0.4986 - val_loss: 7.5266 - val_binary_accuracy: 0.5046
Epoch 7/10
1094/1094 [======